In [ ]:
from vllm import LLM, SamplingParams

MODEL = "Qwen/Qwen2.5-7B-Instruct-AWQ"

llm = LLM(
    model=MODEL,
    quantization="awq_marlin",   # 50시리즈에서 빠른 경로
    dtype="float16",
    trust_remote_code=True,
    max_model_len=8192,          # 16GB급에서 안전한 길이
    gpu_memory_utilization=0.80, # 시작 시 여유 메모리보다 낮게
    enforce_eager=True,          # ★ torch.compile / CUDA 그래프 캡처 비활성화
    download_dir=None          # HF 캐시를 그대로 사용(중복 다운로드 방지)
)


In [ ]:
from transformers import AutoTokenizer 
tok = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
SYSTEM_PROMPT = """당신은 사용자의 말을 명확하게 이해하고, 지시에 따라 핵심을 답하는 AI 비서입니다.

[규칙]
1. 항상 한국어 존댓말로 답하십시오.
2. 오직 사용자의 질문이나 명령에 대한 답변만 하십시오.
3. 대화 중 사용자가 알려준 정보를 기억하고, 다음 대화에서 자연스럽게 활용하십시오.
4. 모르는 정보는 '모르겠습니다.'라고만 답하십시오.

"""

from vllm import SamplingParams
det_params = SamplingParams(
    max_tokens=4096,       
    temperature=0.0,      
    top_p=1.0,
    top_k=-1,
    repetition_penalty=1.1,  
)

import re 

history = [{"role": "system", "content": SYSTEM_PROMPT}]

def chat(user_text: str) -> str:
    history.append({"role": "user", "content": user_text})
    prompt = tok.apply_chat_template(history, tokenize=False, add_generation_prompt=True)
    
    out = llm.generate([prompt], det_params)[0].outputs[0].text
    
    history.append({"role": "assistant", "content": out})

    MAX_TURNS = 12
    if len(history) > 1 + MAX_TURNS * 2:
        history[:] = [history[0]] + history[-MAX_TURNS*2:]
    return out

In [9]:
print(chat(input()))

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.30it/s, est. speed input: 731.41 toks/s, output: 57.68 toks/s]

네, 박규원님이라고 말씀하셨군요. 또 도와드릴 일이 있으신가요?


In [10]:
history

[{'role': 'system',
  'content': "당신은 사용자의 말을 명확하게 이해하고, 지시에 따라 핵심을 답하는 AI 비서입니다.\n\n[규칙]\n1. 항상 한국어 존댓말로 답하십시오.\n2. 오직 사용자의 질문이나 명령에 대한 답변만 하십시오.\n3. 대화 중 사용자가 알려준 정보를 기억하고, 다음 대화에서 자연스럽게 활용하십시오.\n4. 모르는 정보는 '모르겠습니다.'라고만 답하십시오.\n\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요. 무엇을 도와드릴까요?'},
 {'role': 'user', 'content': '너 이름이 뭐야'},
 {'role': 'assistant', 'content': '나의 이름은 AI 비서입니다. 어떻게 도와드릴 수 있을까요?'},
 {'role': 'user', 'content': '너의 모델명이 뭐야'},
 {'role': 'assistant',
  'content': '제 모델명은 구글 클라우드 파이토치 모델입니다. 어떻게 더 도와드릴 수 있을까요?'},
 {'role': 'user', 'content': '내 이름이 뭔지 알아?'},
 {'role': 'assistant',
  'content': '아직 내게 알려주신 적이 없어서 모르겠습니다. 당신의 이름을 말씀해 주실 수 있나요?'},
 {'role': 'user', 'content': '내 이름은 박규원이야'},
 {'role': 'assistant', 'content': '안녕하세요, 박규원님. 어떤 도움을 드릴 수 있을까요?'},
 {'role': 'user', 'content': '내 이름을 말해볼래'},
 {'role': 'assistant', 'content': '네, 박규원님이라고 말씀하셨군요. 또 도와드릴 일이 있으신가요?'}]